*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
import csv
X_train = []
y_train = []
X_test = []
y_test = []
tsv_file = open(train_file_path)
read_tsv = csv.reader(tsv_file, delimiter="\t")

for txt in read_tsv:
  if txt[0] == "ham":
    y_train.append(0)
  else:
    y_train.append(1)
  X_train.append(txt[1])

tsv_file = open(test_file_path)
read_tsv = csv.reader(tsv_file, delimiter="\t")

for txt in read_tsv:
  if txt[0] == "ham":
    y_test.append(0)
  else:
    y_test.append(1)
  X_test.append(txt[1])

In [ ]:
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=7700, split=' ')
tokenizer.fit_on_texts(X_train)
print(tokenizer.word_index)
X_tokens = tokenizer.texts_to_sequences(X_train)
X_pad = pad_sequences(X_tokens)
print(X_pad[0])

In [ ]:
model = keras.Sequential()

model.add(tf.keras.layers.Embedding(7700,32))
model.add(tf.keras.layers.LSTM(32))
model.add(tf.keras.layers.Dense(1,'sigmoid'))

In [ ]:
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [ ]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
model.fit(X_pad,y_train,epochs=6,validation_split=0.2)

In [ ]:

word_index = tokenizer.word_index

def encode(text):
  txt_tokens = keras.preprocessing.text.text_to_word_sequence(text)
  txt_tokens = [word_index[word] if word in word_index else 0 for word in txt_tokens]
  return(pad_sequences([txt_tokens],250)[0])

encoded = encode("you can never do nothing")
encoded

In [ ]:
reverse_word_index = {value : key for (key,value) in word_index.items()}
def decode(integers):
  PAD = 0
  text = ""
  for num in integers:
    if num != PAD:
      text += reverse_word_index[num]+" "
  return text[:-1]

print(decode(encoded))

In [ ]:
def predict(text):
  encoded_text = encode(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred)
  return(result)

predict("I loved this movie")

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(text):
  encoded_text = encode(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  prediction = model.predict(pred)[0]
  label = ""
  if prediction <= 0.1:
    label = "ham"
  else:
    label = "spam"
  
  result = []
  result.append(prediction[0])
  result.append(label)
  return result

test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"]
                
for msg in test_messages:
  prediction = predict_message(msg)
  print(prediction)  

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
